In [ ]:
import os
from time import time
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import cv2
import random

import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, Dense, MaxPooling2D, Flatten
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

import torch
from torch.nn import Linear, CrossEntropyLoss, Dropout
from torch.optim import Adam
from torch.utils.data import DataLoader

import torchvision
from torchvision.datasets import ImageFolder
from torchvision.models import resnet18
from torchvision.transforms import transforms


# Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
tfm = transforms.Compose([
    transforms.Resize((224, 224), interpolation=2),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [ ]:
train_path = '/content/drive/My Drive/mini_project/dataset/split/train'
test_path = '/content/drive/My Drive/mini_project/dataset/split/test'

In [ ]:


train = ImageFolder(train_path, transform=tfm)
test = ImageFolder(test_path, transform=tfm)

In [ ]:
train_loader = DataLoader(train, batch_size=10, shuffle=True)
test_loader = DataLoader(test, batch_size=10, shuffle=True)

In [ ]:
model = resnet18(pretrained=True)
model.fc = Linear(in_features=512, out_features=3)

/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

In [ ]:
model.fc = Dropout(0.2)

In [ ]:
model = model.to(device)

In [ ]:
optimiser = Adam(model.parameters(), lr=3e-4, weight_decay=0.0001)
loss_fn = CrossEntropyLoss()

In [ ]:
preds = []

for epoch in range(10):
    preds = []
    start = time()
    
    tr_acc = 0
    test_acc = 0
    
    # Train
    model.train()
    
    with tqdm(train_loader, unit="batch") as tepoch:
        # in each minibatch
        for xtrain, ytrain in tepoch:
            # clear the gradient
            optimiser.zero_grad()
            xtrain = xtrain.to(device)
            # compute model output
            train_prob = model(xtrain)
            train_prob = train_prob.cpu()
            
            loss = loss_fn(train_prob, ytrain)
            # initialise weights for last layer
            loss.backward()
            # update model weights based on last layer, towards first
            optimiser.step()
            
            # training ends
            
            train_pred = torch.max(train_prob, 1).indices
            tr_acc += int(torch.sum(train_pred == ytrain))
            
        ep_tr_acc = tr_acc / len(train)
    
    # Evaluate
    model.eval()
    with torch.no_grad():
        for xtest, ytest in test_loader:
            xtest = xtest.to(device)
            test_prob = model(xtest)
            test_prob = test_prob.cpu()
            preds.append([test_prob, ytest])
            
            test_pred = torch.max(test_prob,1).indices
            test_acc += int(torch.sum(test_pred == ytest))
            
        ep_test_acc = test_acc / len(test)
    
    end = time()
    duration = (end - start) / 60
    
    print(f"Epoch: {epoch+1}, Time: {duration}, Loss: {loss}\nTrain_acc: {ep_tr_acc}, Test_acc: {ep_test_acc}")

  2%|▏         | 2/120 [00:11<11:15,  5.73s/batch]


KeyboardInterrupt: ignored